## Data mining questions

1- Why a simple linear regression model (without any activation function) is not good for classification task, compared to Perceptron/Logistic regression?

- because linear regression is usually used to predict continuous values but classification is used to predict discrete values, and linear regression is senstive to outliers
- logistic regression is usually used for classification task and A perceptron's used as binary classifier like logistic regression but the difference is that perceptron uses step activation function.
- both logistic regression and preceptron are better than linear regression in classification task.

---


2- What's a decision tree and how it is different to a logistic regression model?

- A decision tree is a supervised machine learning that categorizes or predicts outcomes based on the answers to a previous set of questions.

- It has a tree-like structure with branches and nodes that represent different outcomes or decisions

The difference between decision tree and logistic regression is that 
- logistic regression assumes that the data is linearly separable in space
- decision tree is a*non-linear classifier
- Decision trees can handle categorical data better, while logistic regression can handle continuous data better

---


3- What's the difference between grid search and random search?

- Grid search tries every possible hyperparameter combination to find the best one, but it is very slow to run
- Random search randomly picks hyperparameter combinations from a grid and evaluates the model performance. It is faster than grid search, but it may not find the optimal hyperparameters

---

4- What's the difference between bayesian search and random search?

- Bayesian search is an optimization algorithm that sequentially uses a model-based approach to determine the next hyperparameter value based on the results of the previous iteration.
- Random search uses a grid of hyperparameter values and selects random combinations for training and scoring the model.
- In terms of finding the best hyperparameters, Bayesian search outperforms random search and is faster than grid search

## problem formulation
</br>

### problem definition

- building a machine learning model to predict the probability (0-1, float) (output) that the dating session will match. 

- inputs : (191 features) for both training set (5909 observations) and test set (2469 observations) 

- output : (1 feature) for train set (5909 observations) as output.

---

### Data mining function
- classification and prediction

---

### Challenges

- that our dataset has a lot of null values 

- that the is an imbalanced dataset a lot 0's and less of 1's


---

### Model impact

- It help people to meet their suitable partners 

---

### The ideal solution

- Using a XGBClassifierr with grid search to search for the best hyperparameters combinations to get better ROC-AUC result (88.6%)


## Experimental protocol

### Data preprocessing
---
- building a pipeline which will do some preprocessing for the features.
- using imputer with mean strategy for numeric values and constant for categorical values to impute missing values 
- using standard scaling

---
### Building models

- creating search spaces (random, bayesian and grid)
- training each model many times with different hyperparameters and choose the best
- Start predicting the test data

At the end i tried deep nural network with hyperparameter tunning but the accuracy was a bit lower than xgboost it was about 0.85166 and the xgb was 0.886 on public leaderboard

In [5]:
!pip install scikit-optimize

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 KB 3.3 MB/s eta 0:00:00


In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler, OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from xgboost.sklearn import XGBClassifier
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn import metrics
from sklearn.model_selection import train_test_split
import re
import math
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

Reading files

In [8]:
df= pd.read_csv('train.csv',index_col='id')
dftest=pd.read_csv('test.csv',index_col='id')

df

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2583,0,3,2,14,18,2,2.0,14,12,372.0,0,-0.03,0,27.0,2.0,30.00,15.00,15.00,20.00,5.00,15.00,7.0,7.0,7.0,6.0,5.0,NaN,7.0,1.0,2.0,33.0,Ed.D. in higher education policy at TC,9.0,University of Michigan-Ann Arbor,"1,290.00","21,645.00",3.0,2.0,1.0,"Palo Alto, CA",NaN,NaN,1.0,6.0,3.0,University President,2.0,3.0,4.0,4.0,10.0,9.0,10.0,1.0,1.0,8.0,7.0,10.0,8.0,10.0,1.0,4.0,10.0,4.0,2.0,NaN,40.00,15.00,25.00,5.00,15.00,0.00,55.0,10.0,5.0,15.0,5.0,10.0,70.00,5.00,5.00,15.00,5.00,0.00,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,6.0,5.0,7.0,6.0,6.0,NaN,6.0,4.0,0.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,60.00,10.00,10.00,10.00,10.00,0.00,45.0,25.0,5.0,25.0,0.0,0.0,65.00,5.00,5.00,25.00,0.00,0.00,10.0,10.0,10.0,9.0,10.0,10.0,9.0,10.0,9.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6830,1,14,1,3,10,2,NaN,8,8,63.0,0,0.21,0,24.0,4.0,5.00,15.00,45.00,25.00,0.00,10.00,3.0,8.0,5.0,3.0,7.0,1.0,1.0,3.0,2.0,22.0,Engineering,5.0,NaN,NaN,NaN,2.0,8.0,1.0,"Boston, MA","2,021",NaN,5.0,6.0,1.0,Engineer or iBanker or consultant,7.0,8.0,3.0,7.0,10.0,6.0,6.0,7.0,2.0,9.0,5.0,6.0,7.0,7.0,6.0,8.0,8.0,4.0,7.0,9.0,21.00,20.00,19.00,20.00,10.00,10.00,NaN,NaN,NaN,NaN,NaN,NaN,30.00,15.00,10.00,20.00,10.00,15.00,7.0,9.0,8.0,8.0,8.0,NaN,NaN,NaN,NaN,NaN,5.0,8.0,8.0,5.0,7.0,6.0,6.0,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,22.00,17.00,17.00,17.00,10.00,17.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,7.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,20.00,20.00,15.00,20.00,10.00,15.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,8.0,8.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN
4840,1,14,1,13,10,8,8.0,10,10,331.0,0,0.43,0,34.0,2.0,15.00,15.00,10.00,25.00,10.00,25.00,4.0,8.0,7.0,4.0,7.0,3.0,3.0,2.0,2.0,27.0,Urban Planning,5.0,"Rizvi College of Architecture, Bombay University",NaN,NaN,6.0,1.0,1.0,"Bombay, India",NaN,NaN,1.0,4.0,2.0,Real Estate Consulting,7.0,4.0,2.0,7.0,6.0,8.0,8.0,8.0,6.0,6.0,8.0,4.0,10.0,10.0,7.0,7.0,5.0,10.0,2.0,NaN,10.00,20.00,25.00,20.00,20.00,5.00,25.0,5.0,20.0,25.0,10.0,15.0,25.00,10.00,20.00,25.00,10.00,10.00,7.0,9.0,7.0,9.0,10.0,7.0,9.0,9.0,7.0,9.0,6.0,5.0,7.0,5.0,7.0,6.0,6.0,5.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN

# Data Exploration

In [9]:
#showing null values
df.isnull().sum().sort_values(ascending=False)

num_in_3    5449
numdat_3    4849
expnum      4627
amb7_2      4519
sinc7_2     4519
shar7_2     4505
attr7_2     4498
intel7_2    4498
fun7_2      4498
shar7_3     4496
attr7_3     4496
sinc7_3     4496
intel7_3    4496
fun7_3      4496
amb7_3      4496
amb5_3      4496
shar2_3     4496
intel5_3    4496
attr5_3     4496
fun5_3      4496
sinc5_3     4496
amb2_3      3838
fun2_3      3838
intel2_3    3838
attr4_3     3838
sinc2_3     3838
attr2_3     3838
intel4_3    3838
fun4_3      3838
amb4_3      3838
shar4_3     3838
sinc4_3     3838
mn_sat      3674
tuition     3365
date_3      3105
attr1_3     3105
intel3_3    3105
fun3_3      3105
you_call    3105
them_cal    3105
fun1_3      3105
amb3_3      3105
sinc1_3     3105
intel1_3    3105
amb1_3      3105
shar1_3     3105
sinc3_3     3105
attr3_3     3105
amb3_s      3069
fun3_s      3069
intel3_s    3069
sinc3_s     3069
attr3_s     3069
intel1_s    2994
attr1_s     2994
sinc1_s     2994
shar1_s     2994
amb1_s      2994
fun1_s      29

as we can see the data is full of null values that need handling and filling

In [10]:

#dropping pid column since we already have an id
df= df.drop(['pid'],axis=1)
dftest=dftest.drop(['pid'],axis=1)
df

,gender,idg,condtn,wave,round,position,positin1,order,partner,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2583,0,3,2,14,18,2,2.0,14,12,0,-0.03,0,27.0,2.0,30.00,15.00,15.00,20.00,5.00,15.00,7.0,7.0,7.0,6.0,5.0,NaN,7.0,1.0,2.0,33.0,Ed.D. in higher education policy at TC,9.0,University of Michigan-Ann Arbor,"1,290.00","21,645.00",3.0,2.0,1.0,"Palo Alto, CA",NaN,NaN,1.0,6.0,3.0,University President,2.0,3.0,4.0,4.0,10.0,9.0,10.0,1.0,1.0,8.0,7.0,10.0,8.0,10.0,1.0,4.0,10.0,4.0,2.0,NaN,40.00,15.00,25.00,5.00,15.00,0.00,55.0,10.0,5.0,15.0,5.0,10.0,70.00,5.00,5.00,15.00,5.00,0.00,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,6.0,5.0,7.0,6.0,6.0,NaN,6.0,4.0,0.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,60.00,10.00,10.00,10.00,10.00,0.00,45.0,25.0,5.0,25.0,0.0,0.0,65.00,5.00,5.00,25.00,0.00,0.00,10.0,10.0,10.0,9.0,10.0,10.0,9.0,10.0,9.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6830,1,14,1,3,10,2,NaN,8,8,0,0.21,0,24.0,4.0,5.00,15.00,45.00,25.00,0.00,10.00,3.0,8.0,5.0,3.0,7.0,1.0,1.0,3.0,2.0,22.0,Engineering,5.0,NaN,NaN,NaN,2.0,8.0,1.0,"Boston, MA","2,021",NaN,5.0,6.0,1.0,Engineer or iBanker or consultant,7.0,8.0,3.0,7.0,10.0,6.0,6.0,7.0,2.0,9.0,5.0,6.0,7.0,7.0,6.0,8.0,8.0,4.0,7.0,9.0,21.00,20.00,19.00,20.00,10.00,10.00,NaN,NaN,NaN,NaN,NaN,NaN,30.00,15.00,10.00,20.00,10.00,15.00,7.0,9.0,8.0,8.0,8.0,NaN,NaN,NaN,NaN,NaN,5.0,8.0,8.0,5.0,7.0,6.0,6.0,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,22.00,17.00,17.00,17.00,10.00,17.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,7.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,20.00,20.00,15.00,20.00,10.00,15.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,8.0,8.0,7.0,8.0,NaN,NaN,NaN,NaN,NaN
4840,1,14,1,13,10,8,8.0,10,10,0,0.43,0,34.0,2.0,15.00,15.00,10.00,25.00,10.00,25.00,4.0,8.0,7.0,4.0,7.0,3.0,3.0,2.0,2.0,27.0,Urban Planning,5.0,"Rizvi College of Architecture, Bombay University",NaN,NaN,6.0,1.0,1.0,"Bombay, India",NaN,NaN,1.0,4.0,2.0,Real Estate Consulting,7.0,4.0,2.0,7.0,6.0,8.0,8.0,8.0,6.0,6.0,8.0,4.0,10.0,10.0,7.0,7.0,5.0,10.0,2.0,NaN,10.00,20.00,25.00,20.00,20.00,5.00,25.0,5.0,20.0,25.0,10.0,15.0,25.00,10.00,20.00,25.00,10.00,10.00,7.0,9.0,7.0,9.0,10.0,7.0,9.0,9.0,7.0,9.0,6.0,5.0,7.0,5.0,7.0,6.0,6.0,5.0,0.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6

figuring out the datatypes of each column to know which to convert to category

In [11]:
df.dtypes.value_counts()

float64    172
int64       10
object       8
dtype: int64

In [12]:
#categorical features
df.select_dtypes(include=['object'])

,field,undergra,mn_sat,tuition,from,zipcode,income,career
id,,,,,,,,
2583,Ed.D. in higher education policy at TC,University of Michigan-Ann Arbor,"1,290.00","21,645.00","Palo Alto, CA",NaN,NaN,University President
6830,Engineering,NaN,NaN,NaN,"Boston, MA","2,021",NaN,Engineer or iBanker or consultant
4840,Urban Planning,"Rizvi College of Architecture, Bombay University",NaN,NaN,"Bombay, India",NaN,NaN,Real Estate Consulting
5508,International Affairs,NaN,NaN,NaN,"Washington, DC","10,471","45,300.00",public service
4828,Business,Harvard College,"1,400.00","26,019.00",Midwest USA,"66,208","46,138.00",undecided
...,...,...,...,...,...,...,...,...
3390,Clinical Psychology,NaN,NaN,NaN,New York,"11,803","65,708.00",Psychologist
4130,MBA,NaN,NaN,NaN,Colombia,NaN,NaN,Consulting
1178,MA Science Education,University of Washington,"1,155.00","13,258.00",Seattle,"98,115","37,881.00",Teacher


Some numerical columns have a comma seperator in the middle that misidentifies them as Strings so we will have to remove the comma and convert their data type

In [13]:
#some columns have ',' in middle of number which makes it count as a string
df['mn_sat'].value_counts()

1,400.00    287
1,430.00    194
1,290.00    129
1,450.00    116
1,309.00    102
1,340.00     99
1,260.00     89
1,360.00     79
1,460.00     78
1,330.00     59
1,310.00     59
1,188.00     55
1,210.00     43
1,380.00     41
1,250.00     39
1,030.00     34
1,130.00     34
1,258.00     32
1,320.00     31
1,160.00     29
1,200.00     24
1,110.00     23
1,410.00     22
1,206.00     22
1,242.00     22
1,155.00     21
1,402.00     20
1,308.00     18
1,490.00     17
1,227.00     17
990          16
1,149.00     16
1,365.00     16
1,011.00     15
1,105.00     14
1,090.00     14
1,100.00     14
1,157.00     14
1,060.00     14
1,341.00     14
1,239.00     14
1,014.00     13
1,220.00     13
1,180.00     12
1,159.00     12
1,280.00     12
1,212.00     12
1,092.00     12
1,370.00     11
1,185.00     11
1,331.00     11
1,050.00     11
1,034.00     11
1,140.00     10
1,178.00      8
1,214.00      7
1,070.00      7
1,270.00      7
914           7
1,097.00      7
1,470.00      7
1,191.00      6
1,267.00

In [14]:
#remove string
df['mn_sat'] = df['mn_sat'].str.replace((","), (""))
dftest['mn_sat'] = dftest['mn_sat'].str.replace((","), (""))
#convert to float
df["mn_sat"] = df["mn_sat"].astype(float)
dftest["mn_sat"] = dftest["mn_sat"].astype(float)
df['mn_sat'].value_counts()

1400.0    287
1430.0    194
1290.0    129
1450.0    116
1309.0    102
1340.0     99
1260.0     89
1360.0     79
1460.0     78
1330.0     59
1310.0     59
1188.0     55
1210.0     43
1380.0     41
1250.0     39
1030.0     34
1130.0     34
1258.0     32
1320.0     31
1160.0     29
1200.0     24
1110.0     23
1410.0     22
1206.0     22
1242.0     22
1155.0     21
1402.0     20
1308.0     18
1490.0     17
1227.0     17
990.0      16
1149.0     16
1365.0     16
1011.0     15
1105.0     14
1090.0     14
1100.0     14
1157.0     14
1060.0     14
1341.0     14
1239.0     14
1014.0     13
1220.0     13
1180.0     12
1159.0     12
1280.0     12
1212.0     12
1092.0     12
1370.0     11
1185.0     11
1331.0     11
1050.0     11
1034.0     11
1140.0     10
1178.0      8
1214.0      7
1070.0      7
1270.0      7
914.0       7
1097.0      7
1470.0      7
1191.0      6
1267.0      6
1080.0      6
1230.0      5
1099.0      5
1134.0      5
1215.0      5
Name: mn_sat, dtype: int64

In [15]:
#has same country names but lower case
df['from'].value_counts(dropna=False).sort_values()

Greenwich, CT                                                   2
Europe                                                          2
sofia, bg                                                       3
Upstate New York                                                3
Pougkeepsie NY                                                  4
china                                                           4
Boston, Ma                                                      5
lOS aNGELES                                                     5
Greece/Germany                                                  5
Santa Barbara, California                                       5
USA/American                                                    5
Lexington, MA                                                   5
brooklyn, ny                                                    5
TN                                                              5
Puerto Rico                                                     5
San Diego,

In [16]:
df['from'] = df['from'].str.upper()
dftest['from'] = dftest['from'].str.upper()
df['from'].value_counts().sort_values()

GREENWICH, CT                                                   2
EUROPE                                                          2
UPSTATE NEW YORK                                                3
SOFIA, BG                                                       3
POUGKEEPSIE NY                                                  4
WESTCHESTER, NEW YORK                                           5
LEXINGTON, MA                                                   5
MANILA, PHILIPPINES                                             5
PUERTO RICO                                                     5
GREECE/GERMANY                                                  5
USA/AMERICAN                                                    5
SANTA BARBARA, CALIFORNIA                                       5
SAN DIEGO, CA                                                   5
TN                                                              5
NEBRASKA                                                        6
PITTSBURGH

In [17]:
#replace commas
df['income'] = df['income'].str.replace((","), (""))
dftest['income'] = dftest['income'].str.replace((","), (""))
#convert it to float
df["income"] = df["income"].astype(float)
dftest["income"] = dftest["income"].astype(float)

df['income'].value_counts()

55080.0     82
78704.0     30
53229.0     30
49409.0     28
25401.0     25
33772.0     25
80006.0     22
69487.0     21
52010.0     20
16767.0     20
64716.0     19
48944.0     18
25786.0     18
49642.0     18
72412.0     18
62997.0     18
65708.0     18
35187.0     18
61686.0     18
20000.0     17
36673.0     17
55550.0     17
39123.0     17
15863.0     17
66827.0     17
52805.0     17
26706.0     17
39919.0     17
35867.0     17
73073.0     16
47005.0     16
55138.0     16
48137.0     16
51725.0     16
46792.0     16
45300.0     16
53501.0     16
32508.0     16
109031.0    16
41191.0     16
36510.0     16
31809.0     15
53196.0     15
35968.0     15
46153.0     15
26298.0     15
71787.0     15
50572.0     15
58802.0     15
50501.0     15
52804.0     15
62875.0     14
53771.0     14
76624.0     14
41778.0     14
47624.0     14
80608.0     14
18283.0     14
27503.0     14
65693.0     14
30038.0     14
36223.0     14
40244.0     14
37881.0     14
28759.0     14
27248.0     14
56118.0   

In [18]:
df['tuition'] = df['tuition'].str.replace((","), (""))
dftest['tuition'] = dftest['tuition'].str.replace((","), (""))
#convert it to float
df["tuition"] = df["tuition"].astype(float)
dftest["tuition"] = dftest["tuition"].astype(float)

df['tuition'].value_counts()

26908.0    174
26019.0    131
15162.0    102
25380.0     78
26062.0     73
26100.0     67
26892.0     67
21645.0     65
27395.0     64
26630.0     58
11311.0     55
15309.0     49
27230.0     48
24742.0     45
33585.0     43
20100.0     34
25917.0     34
25425.0     34
25020.0     32
15260.0     32
25533.0     31
23500.0     29
26170.0     27
26377.0     26
27100.0     26
14915.0     24
10450.0     23
12900.0     23
22481.0     22
17125.0     22
10704.0     22
13258.0     21
14388.0     21
27240.0     20
24036.0     19
22170.0     19
26562.0     18
14433.0     18
25555.0     18
16650.0     17
17030.0     17
15990.0     17
27728.0     17
34290.0     17
9811.0      17
15004.0     16
10096.0     16
27025.0     16
25026.0     16
23187.0     16
9162.0      16
25552.0     15
25335.0     15
9729.0      15
11360.0     14
26376.0     14
27982.0     14
26580.0     14
26871.0     14
25504.0     14
14493.0     14
11424.0     14
20337.0     14
13413.0     13
8421.0      13
13046.0     13
21960.0   

In [19]:
df['zipcode'] = df['zipcode'].str.replace((","), (""))
dftest['zipcode'] = dftest['zipcode'].str.replace((","), (""))
#convert it to float
df["zipcode"] = df["zipcode"].astype(float)
dftest["zipcode"] = dftest["zipcode"].astype(float)

df['zipcode'].value_counts()

0.0        244
10021.0     94
10027.0     88
10025.0     77
10012.0     31
          ... 
20854.0      3
90620.0      3
10803.0      2
20853.0      2
11020.0      2
Name: zipcode, Length: 409, dtype: int64

In [ ]:
X_test['career'].map(type).nunique()

1

Now in order for us to pass the categorical features to the pipeline we have to convert the data type of the columns from "Object" to "Category"

In [20]:
#convert all object features to categories for train and
df.loc[:, df.dtypes == 'object'] =\
    df.select_dtypes(['object'])\
    .apply(lambda x: x.astype('category'))

dftest.loc[:, dftest.dtypes == 'object'] =\
    dftest.select_dtypes(['object'])\
    .apply(lambda x: x.astype('category'))

df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5909 entries, 2583 to 8149
Columns: 190 entries, gender to amb5_3
dtypes: category(4), float64(176), int64(10)
memory usage: 8.5 MB


after data cleaning we now have 4 Categorical features

In [21]:
df.loc[:, df.dtypes == 'category'].nunique()

field       259
undergra    241
from        244
career      367
dtype: int64

splitting the data and viewing the shape before entering the modeling phase

In [22]:
y_train = df['match'] # lower case for vector
X_train = df.drop('match', axis=1) # upper case for matrix
X_test=dftest
print('original shape', X_train.shape, y_train.shape,X_test.shape)

original shape (5909, 189) (5909,) (2469, 189)


# Pipeline building
one of the Advantages of a tunable pipeline is the ability to run and tune multiple models in a shorter amount of time and orchestrate data transfoming and model buiding without the need of hard coding.

In [23]:
# we extract numeric features and categorical features names
# for later use

# numeric features can be selected by: (based on the df2.info() output )
features_numeric = list(X_train.select_dtypes(include=['float64', 'int64']))

# categorical features can be selected by: (based on the df2.info() output )
features_categorical = list(X_train.select_dtypes(include=['category']))

print('numeric features:', features_numeric)
print('categorical features:', features_categorical)

numeric features: ['gender', 'idg', 'condtn', 'wave', 'round', 'position', 'positin1', 'order', 'partner', 'int_corr', 'samerace', 'age_o', 'race_o', 'pf_o_att', 'pf_o_sin', 'pf_o_int', 'pf_o_fun', 'pf_o_amb', 'pf_o_sha', 'attr_o', 'sinc_o', 'intel_o', 'fun_o', 'amb_o', 'shar_o', 'like_o', 'prob_o', 'met_o', 'age', 'field_cd', 'mn_sat', 'tuition', 'race', 'imprace', 'imprelig', 'zipcode', 'income', 'goal', 'date', 'go_out', 'career_c', 'sports', 'tvsports', 'exercise', 'dining', 'museums', 'art', 'hiking', 'gaming', 'clubbing', 'reading', 'tv', 'theater', 'movies', 'concerts', 'music', 'shopping', 'yoga', 'exphappy', 'expnum', 'attr1_1', 'sinc1_1', 'intel1_1', 'fun1_1', 'amb1_1', 'shar1_1', 'attr4_1', 'sinc4_1', 'intel4_1', 'fun4_1', 'amb4_1', 'shar4_1', 'attr2_1', 'sinc2_1', 'intel2_1', 'fun2_1', 'amb2_1', 'shar2_1', 'attr3_1', 'sinc3_1', 'fun3_1', 'intel3_1', 'amb3_1', 'attr5_1', 'sinc5_1', 'intel5_1', 'fun5_1', 'amb5_1', 'attr', 'sinc', 'intel', 'fun', 'amb', 'shar', 'like', 'prob',

* for preprocessing numerical data: we will use:

Simple Imputer to handle missing data and Standard Scaler for Standararization

* for preprocessing categorical data: we will use:

Simple Imputer to handle missing data and on hot encoder for encoding

* We will try 3 different models each with 3 different Search algorithms which makes 9 trials in total

In [24]:
np.random.seed(0)

# define a pipe line for numeric feature preprocessing
transformer_numeric = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())]
)

# define a pipe line for categorical feature preprocessing
transformer_categorical = Pipeline(
    steps=[
        ('imputer', SimpleImputer(strategy='constant')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]
)
# define the preprocessor 

preprocessor = ColumnTransformer(
    transformers=[
        ('num', transformer_numeric, features_numeric),
        ('cat', transformer_categorical, features_categorical)
    ]
    
)

full_piplineLOG = Pipeline(  
    steps=[
        ('preprocessor', preprocessor), 
        ('my_classifier', 
           LogisticRegression(), # Logistic Reggression.
        )
    ]
)
full_piplineLOG
full_piplineXGB = Pipeline(  
    steps=[
        ('preprocessor', preprocessor), 
        ('my_classifier', 
           XGBClassifier(), # XGBClassifier.
        )
    ]
)
full_piplineXGB
# combine the preprocessor with the model as a full tunable pipeline
# we gave them a name so we can set their hyperparameters
full_piplineRAN = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('my_classifier', 
           RandomForestClassifier(),
        )
    ]
)
full_piplineRAN

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['gender', 'idg', 'condtn',
                                                   'wave', 'round', 'position',
                                                   'positin1', 'order',
                                                   'partner', 'int_corr',
                                                   'samerace', 'age_o',
                                                   'race_o', 'pf_o_att',
                                                   'pf_o_sin', 'pf_o_int',
                                                   'pf_o_fun', 'pf_o_amb',
                                                   'pf_o_sha', 'attr_o',
                                                   'sinc_o', 'intel_o', 'fun_o',
                                                   'amb_o', 'shar_o', 'like_o',
                                                   'prob_o', 'met_o', 'age',
                                                   'field_cd', ...]),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='constant')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['field', 'undergra', 'from',
                                                   'career'])])),
                ('my_classifier', RandomForestClassifier())])

# Random Forest
1-Grid Search

In [ ]:
#define Parameters
param_rndf = {
    'preprocessor__num__imputer__strategy': ['mean'],
    # preprocessor__num__imputer__strategy points to preprocessor->num (a Pipeline)-> imputer -> strategy
    'my_classifier__n_estimators': [30, 40,50],  
     # my_classifier__n_estimators points to my_classifier->n_estimators 
    'my_classifier__max_depth':[5,10, 20],
    'my_classifier__max_features': ['auto'],
    'my_classifier__criterion': ['entropy'], 
    'my_classifier__class_weight':['balanced'],
    'my_classifier__min_samples_split':[2,4,6,8]   
}

grid_search = GridSearchCV(
    full_piplineRAN, param_rndf, cv=5, verbose=1, n_jobs=2, 
    scoring='roc_auc')

grid_search.fit(X_train, y_train)

print('best score {}'.format(grid_search.best_score_))
print('best score {}'.format(grid_search.best_params_))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
best score 0.8489825555441055
best score {'my_classifier__class_weight': 'balanced', 'my_classifier__criterion': 'entropy', 'my_classifier__max_depth': 5, 'my_classifier__max_features': 'auto', 'my_classifier__min_samples_split': 8, 'my_classifier__n_estimators': 50, 'preprocessor__num__imputer__strategy': 'mean'}


2-Random search

In [ ]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit

rnd_search = RandomizedSearchCV(
    full_piplineRAN, param_rndf, cv=5, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc')

rnd_search.fit(X_train, y_train)

print('best score {}'.format(rnd_search.best_score_))
print('best score {}'.format(rnd_search.best_params_))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
best score 0.8452118781576494
best score {'preprocessor__num__imputer__strategy': 'mean', 'my_classifier__n_estimators': 50, 'my_classifier__min_samples_split': 2, 'my_classifier__max_features': 'auto', 'my_classifier__max_depth': 10, 'my_classifier__criterion': 'entropy', 'my_classifier__class_weight': 'balanced'}


3-Bayes Search

In [ ]:
rf_param_bayes = {
    'my_classifier__n_estimators': Categorical([40,45,50]),
    'my_classifier__max_depth': Integer(3,50),
    'my_classifier__max_features': Categorical(['sqrt']),
    'my_classifier__criterion': Categorical(['gini', 'entropy']),
     'my_classifier__class_weight': Categorical([None, 'balanced']),
    'my_classifier__min_samples_split':Integer(3,10)
}
baysserc= BayesSearchCV(
    full_piplineRAN, rf_param_bayes, cv=5, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc')
baysserc.fit(X_train, y_train)

print('best score {}'.format(baysserc.best_score_))
print('best score {}'.format(baysserc.best_params_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
best score 0.8442676144498155
best score OrderedDict([('my_classifier__class_weight', None), ('my_classifier__criterion', 'gini'), ('my_classifier__max_depth', 12), ('my_classifier__max_features', 'sqrt'), ('my_classifier__min_samples_split', 10), ('my_classifier__n_estimators', 50)])


Random Forest Observations: 

all search methods have almost the same score with 84% and i updated the parameter space after each search in order to speed up the process and find better solutions

# xgboost
1-Grid Search

In [ ]:
xgb_param = {
  'my_classifier__n_estimators': [425,450,475],
  'my_classifier__max_depth': [12,15,17],
  'my_classifier__learning_rate': [0.05, 0.1, 0.15],
  'my_classifier__gamma': [0],
  'my_classifier__subsample': [0.5,0.7,0.8],
  'my_classifier__class_weight':['balanced']
}

In [ ]:

xgb_grid = GridSearchCV(
    full_piplineXGB, xgb_param, cv=5, verbose=1, n_jobs=-1, 
    scoring='roc_auc')

xgb_grid.fit(X_train, y_train)

print('best score {}'.format(xgb_grid.best_score_))
print('best score {}'.format(xgb_grid.best_params_))

Fitting 5 folds for each of 81 candidates, totalling 405 fits
[01:02:06] WARNING: ../src/learner.cc:767: 
Parameters: { "class_weight" } are not used.

best score 0.8864853192407447
best score {'my_classifier__class_weight': 'balanced', 'my_classifier__gamma': 0, 'my_classifier__learning_rate': 0.05, 'my_classifier__max_depth': 15, 'my_classifier__n_estimators': 475, 'my_classifier__subsample': 0.8}


2-Random Search

In [ ]:
xgb_rnd = RandomizedSearchCV(
    full_piplineXGB, xgb_param, cv=5, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc')

xgb_rnd.fit(X_train, y_train)

print('best score {}'.format(xgb_rnd.best_score_))
print('best score {}'.format(xgb_rnd.best_params_))

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[21:31:58] WARNING: ../src/learner.cc:767: 
Parameters: { "class_weight" } are not used.

best score 0.8861953566123535
best score {'my_classifier__subsample': 0.8, 'my_classifier__n_estimators': 300, 'my_classifier__max_depth': 30, 'my_classifier__learning_rate': 0.05, 'my_classifier__gamma': 0.2, 'my_classifier__class_weight': 'balanced'}


3-Bayes Search

In [ ]:
xgb_param_bayes = {
  'my_classifier__n_estimators': Categorical([100, 200,300]),
  'my_classifier__max_depth': Integer(3,100),
  'my_classifier__learning_rate':Real(1e-5, 0.2, prior='log-uniform'),
  'my_classifier__gamma':Real(1e-5, 0.2, prior='log-uniform'),
  'my_classifier__class_weight': Categorical(['balanced']),
  'my_classifier__subsample':Real(0.5, 1, prior='log-uniform')
}
xgb_bayes= BayesSearchCV(
    full_piplineXGB, xgb_param_bayes, cv=5, verbose=1, n_jobs=-1, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc')
xgb_bayes.fit(X_train, y_train)

print('best score {}'.format(xgb_bayes.best_score_))
print('best score {}'.format(xgb_bayes.best_params_))

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
[20:59:03] WARNING: ../src/learner.cc:767: 
Parameters: { "class_weight" } are not used.

best score 0.8838703465689678
best score OrderedDict([('my_classifier__class_weight', 'balanced'), ('my_classifier__gamma', 5.154316418616202e-05), ('my_classifier__learning_rate', 0.08294098149955924), ('my_classifier__max_depth', 91), ('my_classifier__n_estimators', 200), ('my_classifier__subsample', 0.57983812836977

XGBoost Observations: 

the model had better performance than RF with a score of 88% 

# Logistic Reggression
Logistic reggression is very good in binary classification but normally has problems with very high number of features so we will see how it performs.


1-Grid Search

In [33]:
#define Parameters
param_log = {
    # preprocessor__num__imputer__strategy points to preprocessor->num (a Pipeline)-> imputer -> strategy
    "my_classifier__C": np.logspace(-4, 4, 4),
    'my_classifier__solver' : ['liblinear'],
    'my_classifier__penalty' : ['l1', 'l2']
}

log_grid = GridSearchCV(
    full_piplineLOG, param_log, cv=5, verbose=1, n_jobs=2, 
    scoring='roc_auc')

log_grid.fit(X_train, y_train)

print('best score {}'.format(log_grid.best_score_))
print('best score {}'.format(log_grid.best_params_))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
best score 0.8637069246536828
best score {'my_classifier__C': 0.046415888336127774, 'my_classifier__penalty': 'l2', 'my_classifier__solver': 'liblinear'}


2-Random search

In [35]:
log_rnd = RandomizedSearchCV(
    full_piplineLOG, param_log, cv=5, verbose=1, n_jobs=2, 
    # number of random trials
    n_iter=10,
    scoring='roc_auc')

log_rnd.fit(X_train, y_train)

print('best score {}'.format(log_rnd.best_score_))
print('best score {}'.format(log_rnd.best_params_))

Fitting 5 folds for each of 8 candidates, totalling 40 fits
best score 0.8637069246536828
best score {'my_classifier__solver': 'liblinear', 'my_classifier__penalty': 'l2', 'my_classifier__C': 0.046415888336127774}


Logistic Reggression Observations: 

surprisingly it had decent performance with 86% even better than Random forest with 84% but still underperformed the xgboost with the highest score of 88%

In [ ]:
submission = pd.DataFrame()

submission['id'] = dftest.index

submission['match'] = xgb_grid.predict_proba(X_test)[:,1]

submission.to_csv('grid.csv', index=False)

,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,...,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2
id,,,,,,,,,,,,,,,,,,,,,
2583,0.0,0.046512,1.0,0.65,0.764706,0.047619,0,0.619048,0.523810,0,...,0,0,0,0,0,0,0,0,0,0
6830,1.0,0.302326,0.0,0.10,0.294118,0.047619,1,0.333333,0.333333,1,...,1,1,1,1,1,1,1,1,1,1
4840,1.0,0.302326,0.0,0.60,0.294118,0.333333,2,0.428571,0.428571,2,...,2,2,2,2,2,2,2,2,2,0
5508,1.0,0.860465,1.0,0.40,0.882353,0.809524,3,0.238095,0.285714,3,...,3,3,3,3,3,3,2,3,2,2
4828,1.0,0.534884,1.0,0.65,0.882353,0.238095,4,0.904762,0.761905,4,...,4,4,4,4,2,2,3,2,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3390,0.0,0.000000,1.0,0.40,0.882353,0.047619,0,0.809524,0.000000,29,...,33,40,36,40,38,0,0,0,2,0
4130,1.0,0.534884,1.0,0.40,0.882353,0.857143,8,0.190476,0.238095,402,...,27,37,23,37,35,1,2,3,2,2
1178,0.0,0.279070,1.0,0.50,0.941176,0.190476,7,0.095238,0.809524,31,...,17,18,14,7,19,2,0,2,2,4
